In [ ]:
import pandas as pd
df = pd.read_csv('/content/subset.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))

def extract_nouns(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if not word in stop_words]
    tagged = nltk.pos_tag(tokens)
    nouns = [word for word, pos in tagged if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return nouns

df['nouns'] = df['Positive_Review'].apply(extract_nouns)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
noun_freq = Counter([noun for sublist in df['nouns'] for noun in sublist])

top_nouns = noun_freq.most_common(10)
top_nouns = [noun for noun, freq in top_nouns]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
top_nouns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['staff',
 'location',
 'room',
 'hotel',
 'Location',
 'breakfast',
 'Great',
 'Positive',
 'Staff',
 'rooms']

In [ ]:
df['filtered_nouns'] = df['nouns'].apply(lambda nouns_list: [noun for noun in nouns_list if noun in top_nouns])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df['transactions'] = df.apply(lambda row: [row['Hotel_Name']] + row['filtered_nouns'], axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.f

In [ ]:
hotel_transactions = df['transactions']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
hotel_transactions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0       [Les Plumes Hotel, room, room, Staff, Great, l...
1                                      [Les Plumes Hotel]
2                  [Les Plumes Hotel, Great, hotel, room]
3                          [Les Plumes Hotel, room, room]
4                                [Les Plumes Hotel, room]
                              ...                        
9531                                      [Hotel Atlanta]
9532                            [Hotel Atlanta, Positive]
9533                                      [Hotel Atlanta]
9534                            [Hotel Atlanta, Positive]
9535                           [Hotel Atlanta, breakfast]
Name: transactions, Length: 9536, dtype: object

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit_transform(hotel_transactions)
df_te = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df_te, min_support=0.00001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.00001)

rules['antecedents'] = rules['antecedents'].apply(lambda x: set(x).intersection(set(top_nouns)))
rules = rules[rules['antecedents'].apply(len) > 0]
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,{Great},(Ako Suites Hotel),0.073301,0.013737,0.001363,0.018598,1.353821,0.000356,1.004953,0.282022
2,{Location},(Ako Suites Hotel),0.113045,0.013737,0.001468,0.012987,0.945375,-0.000085,0.999240,-0.061161
5,{Positive},(Ako Suites Hotel),0.072462,0.013737,0.001154,0.015919,1.158803,0.000158,1.002217,0.147747
7,{Staff},(Ako Suites Hotel),0.072043,0.013737,0.001363,0.018923,1.377468,0.000374,1.005285,0.295305
9,{breakfast},(Ako Suites Hotel),0.082110,0.013737,0.000105,0.001277,0.092968,-0.001023,0.987524,-0.914009
...,...,...,...,...,...,...,...,...,...,...
56804,{Staff},"(staff, Great, Location, rooms, breakfast, hot...",0.072043,0.000105,0.000105,0.001456,13.880640,0.000097,1.001353,1.000000
56805,{rooms},"(staff, Great, Location, Staff, breakfast, hot...",0.066590,0.000105,0.000105,0.001575,15.017323,0.000098,1.001472,1.000000
56806,{breakfast},"(staff, Great, Location, Staff, rooms, hotel, ...",0.082110,0.000105,0.000105,0.001277,12.178799,0.000096,1.001174,1.000000
56807,{hotel},"(staff, Great, Location, Staff, rooms, breakfa...",0.188339,0.000105,0.000105,0.000557,5.309577,0.000085,1.000452,1.000000


In [ ]:
# filter
filtered_rules = rules[rules['antecedents'].apply(lambda x: any(noun in x for noun in top_nouns))]
consequents_hotel = df['Hotel_Name'].unique()
single_antecedent_consequent_rules = filtered_rules[(rules['antecedents'].map(len) == 1) &
                                           (rules['consequents'].map(len) == 1) &
                                           (rules['consequents'].apply(lambda x: list(x)[0]).isin(consequents_hotel))]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [ ]:
single_antecedent_consequent_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,{Great},(Ako Suites Hotel),0.073301,0.013737,0.001363,0.018598,1.353821,0.000356,1.004953,0.282022
2,{Location},(Ako Suites Hotel),0.113045,0.013737,0.001468,0.012987,0.945375,-0.000085,0.999240,-0.061161
5,{Positive},(Ako Suites Hotel),0.072462,0.013737,0.001154,0.015919,1.158803,0.000158,1.002217,0.147747
7,{Staff},(Ako Suites Hotel),0.072043,0.013737,0.001363,0.018923,1.377468,0.000374,1.005285,0.295305
9,{breakfast},(Ako Suites Hotel),0.082110,0.013737,0.000105,0.001277,0.092968,-0.001023,0.987524,-0.914009
...,...,...,...,...,...,...,...,...,...,...
631,{hotel},(W London Leicester Square),0.188339,0.027999,0.005348,0.028396,1.014189,0.000075,1.000409,0.017237
633,{location},(W London Leicester Square),0.269820,0.027999,0.007865,0.029149,1.041062,0.000310,1.001184,0.054017
635,{room},(W London Leicester Square),0.200713,0.027999,0.005558,0.027691,0.988983,-0.000062,0.999683,-0.013745
637,{rooms},(W London Leicester Square),0.066590,0.027999,0.002202,0.033071,1.181138,0.000338,1.005245,0.164299


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

dropdown = widgets.Dropdown(options=top_nouns, description='Noun :')


button = widgets.Button(description='Recommend Hotels')

output = widgets.Output()

def recommend_hotels(b):
    with output:
        clear_output(wait=True)
        selected_nouns = {dropdown.value}
        matching_rules = single_antecedent_consequent_rules[single_antecedent_consequent_rules['antecedents'].apply(lambda antecedents: antecedents.issubset(selected_nouns))]
        matching_rules = matching_rules.sort_values(by='confidence', ascending=False)
        if not matching_rules.empty:
            for hotel in matching_rules['consequents'].iloc[0]:
                print(f"Recommended hotel: {hotel}")
        else:
            print("No recommendations available for the selected nouns.")

button.on_click(recommend_hotels)
display(dropdown, button, output)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dropdown(description='Noun :', options=('staff', 'location', 'room', 'hotel', 'Location', 'breakfast', 'Great'…

Button(description='Recommend Hotels', style=ButtonStyle())

Output()